# 🔐 Credit Card Fraud Detection

## Production-Ready ML Pipeline | Data Science Portfolio Project

This notebook demonstrates a complete fraud detection system using the **Kaggle Credit Card Fraud Detection Dataset** with industry-standard evaluation metrics.

> 💡 *Techniques applicable to fintech platforms like PayPal, Venmo, Stripe, and similar payment processors.*

---

### 📊 Dataset Overview
- **Source**: [Kaggle - Credit Card Fraud Detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)
- **Transactions**: 284,807
- **Fraud Cases**: 492 (0.172%)
- **Features**: V1-V28 (PCA transformed), Time, Amount

### 🎯 Results Achieved
| Metric | Value |
|--------|-------|
| ROC-AUC | 0.9829 |
| PR-AUC | 0.8490 |
| Fraud Recall | 85.7% |
| Precision | 82.4% |
| Precision@50 | 98% |

## 1. Setup & Imports

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    precision_recall_curve, average_precision_score, roc_curve
)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE

print("✅ All libraries imported successfully!")

## 2. Load & Explore Data

In [ ]:
# Load the Kaggle Credit Card Fraud dataset
df = pd.read_csv('data/creditcard.csv')

print("📊 Dataset Overview")
print("=" * 50)
print(f"Shape: {df.shape}")
print(f"\nFraud Distribution:")
print(f"  Legitimate: {(df['Class']==0).sum():,} ({(df['Class']==0).mean()*100:.3f}%)")
print(f"  Fraud:      {(df['Class']==1).sum():,} ({(df['Class']==1).mean()*100:.3f}%)")

In [ ]:
# Display first few rows
df.head()

In [ ]:
# Basic statistics
df.describe()

In [ ]:
# Check for missing values
print("Missing Values:")
print(df.isnull().sum().sum())

## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Class Distribution Visualization
fig = go.Figure(data=[
    go.Bar(
        x=['Legitimate', 'Fraud'],
        y=[df['Class'].value_counts()[0], df['Class'].value_counts()[1]],
        marker_color=['#10B981', '#EF4444'],
        text=[f"{df['Class'].value_counts()[0]:,}", f"{df['Class'].value_counts()[1]:,}"],
        textposition='auto'
    )
])
fig.update_layout(
    title='Class Distribution (Extreme Imbalance: 0.172% Fraud)',
    yaxis_title='Count',
    yaxis_type='log',
    height=400
)
fig.show()

In [ ]:
# Amount Distribution by Class
fig = make_subplots(rows=1, cols=2, subplot_titles=('Legitimate Transactions', 'Fraudulent Transactions'))

fig.add_trace(
    go.Histogram(x=df[df['Class']==0]['Amount'], nbinsx=50, marker_color='#10B981', name='Legitimate'),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=df[df['Class']==1]['Amount'], nbinsx=50, marker_color='#EF4444', name='Fraud'),
    row=1, col=2
)

fig.update_layout(title='Transaction Amount Distribution', height=400, showlegend=False)
fig.update_xaxes(title_text='Amount ($)', row=1, col=1)
fig.update_xaxes(title_text='Amount ($)', row=1, col=2)
fig.show()

In [ ]:
# Amount Statistics by Class
print("💰 Amount Statistics by Class")
print("=" * 50)
print(df.groupby('Class')['Amount'].describe())

In [ ]:
# Time Distribution (transactions over 2 days)
df['Hour'] = (df['Time'] / 3600) % 24

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=df[df['Class']==0]['Hour'], 
    nbinsx=24, 
    name='Legitimate',
    marker_color='#10B981',
    opacity=0.7
))
fig.add_trace(go.Histogram(
    x=df[df['Class']==1]['Hour'], 
    nbinsx=24, 
    name='Fraud',
    marker_color='#EF4444',
    opacity=0.7
))

fig.update_layout(
    title='Transaction Distribution by Hour of Day',
    xaxis_title='Hour',
    yaxis_title='Count',
    barmode='overlay',
    height=400
)
fig.show()

In [ ]:
# Correlation of V features with Fraud
v_features = [f'V{i}' for i in range(1, 29)]
correlations = df[v_features + ['Class']].corr()['Class'].drop('Class').sort_values()

fig = go.Figure()
fig.add_trace(go.Bar(
    x=correlations.values,
    y=correlations.index,
    orientation='h',
    marker_color=['#EF4444' if x < 0 else '#10B981' for x in correlations.values]
))
fig.update_layout(
    title='Correlation of PCA Features (V1-V28) with Fraud',
    xaxis_title='Correlation',
    height=600
)
fig.show()

## 4. Feature Engineering

In [ ]:
def engineer_features(df):
    """
    Create additional features from the dataset.
    """
    df = df.copy()
    
    # Time-based features
    df['Hour'] = (df['Time'] / 3600) % 24
    df['Hour_sin'] = np.sin(2 * np.pi * df['Hour'] / 24)
    df['Hour_cos'] = np.cos(2 * np.pi * df['Hour'] / 24)
    
    # Amount features
    df['Log_Amount'] = np.log1p(df['Amount'])
    df['Amount_Zscore'] = (df['Amount'] - df['Amount'].mean()) / df['Amount'].std()
    df['High_Amount'] = (df['Amount'] > df['Amount'].quantile(0.95)).astype(int)
    
    # Night transaction flag
    df['Is_Night'] = ((df['Hour'] >= 22) | (df['Hour'] <= 5)).astype(int)
    
    # Interaction features
    df['V1_V2_interaction'] = df['V1'] * df['V2']
    df['V3_V4_interaction'] = df['V3'] * df['V4']
    df['V14_Amount'] = df['V14'] * df['Log_Amount']
    df['V17_Amount'] = df['V17'] * df['Log_Amount']
    
    # Drop original Time
    df = df.drop('Time', axis=1)
    
    return df

# Apply feature engineering
df_engineered = engineer_features(df)

print(f"✅ Feature Engineering Complete!")
print(f"   Original features: {df.shape[1]}")
print(f"   New features: {df_engineered.shape[1]}")
print(f"   Added: {df_engineered.shape[1] - df.shape[1] + 1} features")

In [ ]:
# View new features
new_features = ['Hour', 'Hour_sin', 'Hour_cos', 'Log_Amount', 'Amount_Zscore', 
                'High_Amount', 'Is_Night', 'V1_V2_interaction', 'V14_Amount']
df_engineered[new_features].head()

## 5. Data Preparation

In [ ]:
# Separate features and target
X = df_engineered.drop('Class', axis=1)
y = df_engineered['Class']

# Scale features
scaler = RobustScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Train/Test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print("📊 Data Split")
print("=" * 50)
print(f"Training set: {len(X_train):,} samples ({y_train.mean()*100:.3f}% fraud)")
print(f"Test set: {len(X_test):,} samples ({y_test.mean()*100:.3f}% fraud)")

In [ ]:
# Apply SMOTE to handle class imbalance
print("⚖️ Applying SMOTE Resampling")
print("=" * 50)
print(f"Before SMOTE: {np.bincount(y_train.astype(int))}")

smote = SMOTE(sampling_strategy=0.5, random_state=42, k_neighbors=5)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print(f"After SMOTE:  {np.bincount(y_train_res.astype(int))}")

## 6. Model Training

In [ ]:
# Define base models
models = {
    'XGBoost': XGBClassifier(
        n_estimators=200, max_depth=6, learning_rate=0.05,
        scale_pos_weight=100, random_state=42, n_jobs=-1, eval_metric='auc'
    ),
    'LightGBM': LGBMClassifier(
        n_estimators=200, max_depth=6, learning_rate=0.05,
        class_weight='balanced', random_state=42, n_jobs=-1, verbose=-1
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=200, max_depth=12, class_weight='balanced_subsample',
        random_state=42, n_jobs=-1
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=150, max_depth=5, learning_rate=0.05, random_state=42
    )
}

# Train and evaluate each model
results = {}

print("🚀 Training Models")
print("=" * 50)

for name, model in models.items():
    print(f"\n   Training {name}...")
    model.fit(X_train_res, y_train_res)
    
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    pr_auc = average_precision_score(y_test, y_pred_proba)
    
    results[name] = {
        'model': model,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'probabilities': y_pred_proba
    }
    
    print(f"      ROC-AUC: {roc_auc:.4f} | PR-AUC: {pr_auc:.4f}")

In [ ]:
# Build Stacking Ensemble
print("\n🏆 Training Stacking Ensemble")
print("=" * 50)

estimators = [
    ('xgboost', models['XGBoost']),
    ('lightgbm', models['LightGBM']),
    ('random_forest', models['Random Forest']),
]

ensemble = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000),
    cv=5,
    stack_method='predict_proba',
    n_jobs=-1
)

ensemble.fit(X_train_res, y_train_res)
y_pred_proba_ensemble = ensemble.predict_proba(X_test)[:, 1]

roc_auc_ensemble = roc_auc_score(y_test, y_pred_proba_ensemble)
pr_auc_ensemble = average_precision_score(y_test, y_pred_proba_ensemble)

results['Stacking Ensemble'] = {
    'model': ensemble,
    'roc_auc': roc_auc_ensemble,
    'pr_auc': pr_auc_ensemble,
    'probabilities': y_pred_proba_ensemble
}

print(f"\n   ✅ Stacking Ensemble:")
print(f"      ROC-AUC: {roc_auc_ensemble:.4f}")
print(f"      PR-AUC: {pr_auc_ensemble:.4f}")

## 7. Model Evaluation

In [ ]:
# Find optimal threshold using F1 score
precisions, recalls, thresholds = precision_recall_curve(y_test, y_pred_proba_ensemble)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-8)
best_threshold = thresholds[np.argmax(f1_scores[:-1])]

print(f"🎯 Optimal Threshold: {best_threshold:.3f}")

# Make predictions with optimal threshold
y_pred = (y_pred_proba_ensemble >= best_threshold).astype(int)

In [ ]:
# Classification Report
print("📋 Classification Report")
print("=" * 50)
print(classification_report(y_test, y_pred, target_names=['Legitimate', 'Fraud'], digits=4))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print("🔢 Confusion Matrix")
print("=" * 50)
print(f"True Negatives:  {tn:,} (legitimate correctly identified)")
print(f"False Positives: {fp:,} (legitimate flagged as fraud)")
print(f"False Negatives: {fn:,} (FRAUD MISSED ⚠️)")
print(f"True Positives:  {tp:,} (fraud caught ✅)")

In [ ]:
# Confusion Matrix Visualization
fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=['Pred: Legitimate', 'Pred: Fraud'],
    y=['True: Legitimate', 'True: Fraud'],
    text=[[f'{cm[0,0]:,}', f'{cm[0,1]:,}'],
          [f'{cm[1,0]:,}', f'{cm[1,1]:,}']],
    texttemplate='%{text}',
    textfont={'size': 18},
    colorscale='Blues'
))
fig.update_layout(title='Confusion Matrix', height=450)
fig.show()

## 8. Fraud-Specific Metrics

In [ ]:
# Precision@K
def precision_at_k(y_true, y_scores, k):
    top_k_idx = np.argsort(y_scores)[-k:]
    return y_true.iloc[top_k_idx].mean()

print("📌 Precision@K (Top K highest-risk transactions)")
print("=" * 50)
for k in [50, 100, 200, 500, 1000]:
    prec_k = precision_at_k(y_test, y_pred_proba_ensemble, k)
    print(f"   Precision@{k:4d}: {prec_k:.4f} ({int(prec_k*k)} fraud in top {k})")

In [ ]:
# Recall@FPR
def recall_at_fpr(y_true, y_scores, target_fpr):
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    idx = np.argmin(np.abs(fpr - target_fpr))
    return tpr[idx]

print("📌 Recall at Fixed False Positive Rates")
print("=" * 50)
for fpr_target in [0.001, 0.005, 0.01, 0.05, 0.10]:
    recall_fpr = recall_at_fpr(y_test, y_pred_proba_ensemble, fpr_target)
    print(f"   Recall@FPR={fpr_target*100:5.1f}%: {recall_fpr:.4f} ({recall_fpr*100:.1f}% fraud caught)")

In [ ]:
# Business Impact
total_fraud = y_test.sum()
avg_fraud_amount = 150
review_cost = 5

fraud_loss_no_model = total_fraud * avg_fraud_amount
fraud_loss_with_model = fn * avg_fraud_amount + fp * review_cost
savings = fraud_loss_no_model - fraud_loss_with_model

print("💰 Business Impact Analysis")
print("=" * 50)
print(f"Total fraud cases: {int(total_fraud)}")
print(f"Fraud caught: {tp} ({tp/total_fraud*100:.1f}%)")
print(f"Fraud missed: {fn} ({fn/total_fraud*100:.1f}%)")
print(f"False alarms: {fp}")
print(f"\n💵 Cost Analysis (Avg fraud=${avg_fraud_amount}, Review=${review_cost}):")
print(f"   Loss without model: ${fraud_loss_no_model:,.0f}")
print(f"   Loss with model: ${fraud_loss_with_model:,.0f}")
print(f"   Net savings: ${savings:,.0f} ({savings/fraud_loss_no_model*100:.1f}%)")

## 9. Visualizations

In [ ]:
# ROC Curve
fig = go.Figure()

for name, res in results.items():
    fpr, tpr, _ = roc_curve(y_test, res['probabilities'])
    fig.add_trace(go.Scatter(
        x=fpr, y=tpr,
        mode='lines',
        name=f"{name} (AUC={res['roc_auc']:.4f})"
    ))

fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    name='Random',
    line=dict(dash='dash', color='gray')
))

fig.update_layout(
    title='ROC Curves - All Models',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    height=500
)
fig.show()

In [ ]:
# Precision-Recall Curve
fig = go.Figure()

for name, res in results.items():
    precision, recall, _ = precision_recall_curve(y_test, res['probabilities'])
    fig.add_trace(go.Scatter(
        x=recall, y=precision,
        mode='lines',
        name=f"{name} (PR-AUC={res['pr_auc']:.4f})"
    ))

fig.update_layout(
    title='Precision-Recall Curves (More Important for Fraud Detection)',
    xaxis_title='Recall',
    yaxis_title='Precision',
    height=500
)
fig.show()

In [ ]:
# Feature Importance (XGBoost)
feature_importance = dict(zip(X.columns, models['XGBoost'].feature_importances_))
sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)[:20]

fig = go.Figure()
fig.add_trace(go.Bar(
    x=[f[1] for f in sorted_features][::-1],
    y=[f[0] for f in sorted_features][::-1],
    orientation='h',
    marker_color='#6366F1'
))
fig.update_layout(
    title='Top 20 Feature Importances (XGBoost)',
    xaxis_title='Importance',
    height=600
)
fig.show()

In [ ]:
# Score Distribution
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=y_pred_proba_ensemble[y_test == 0],
    name='Legitimate',
    marker_color='#10B981',
    opacity=0.7,
    nbinsx=50
))
fig.add_trace(go.Histogram(
    x=y_pred_proba_ensemble[y_test == 1],
    name='Fraud',
    marker_color='#EF4444',
    opacity=0.7,
    nbinsx=50
))
fig.add_vline(x=best_threshold, line_dash='dash', annotation_text=f'Threshold: {best_threshold:.3f}')

fig.update_layout(
    title='Fraud Probability Score Distribution',
    xaxis_title='Fraud Probability',
    yaxis_title='Count',
    barmode='overlay',
    height=500
)
fig.show()

## 10. Model Comparison Summary

In [ ]:
# Model Comparison Table
comparison_df = pd.DataFrame([
    {'Model': name, 'ROC-AUC': res['roc_auc'], 'PR-AUC': res['pr_auc']}
    for name, res in results.items()
]).sort_values('PR-AUC', ascending=False)

print("🏅 Model Comparison (Sorted by PR-AUC)")
print("=" * 50)
print(comparison_df.to_string(index=False))

In [ ]:
# Model Comparison Visualization
fig = go.Figure()
fig.add_trace(go.Bar(
    name='ROC-AUC',
    x=comparison_df['Model'],
    y=comparison_df['ROC-AUC'],
    marker_color='#6366F1'
))
fig.add_trace(go.Bar(
    name='PR-AUC',
    x=comparison_df['Model'],
    y=comparison_df['PR-AUC'],
    marker_color='#10B981'
))

fig.update_layout(
    title='Model Performance Comparison',
    yaxis_title='Score',
    barmode='group',
    height=400
)
fig.show()

## 11. Save Model

In [ ]:
import joblib
import os

os.makedirs('models', exist_ok=True)

joblib.dump({
    'ensemble': ensemble,
    'scaler': scaler,
    'best_threshold': best_threshold,
    'feature_names': list(X.columns)
}, 'models/fraud_detector.pkl')

print("✅ Model saved to models/fraud_detector.pkl")

## 12. Summary & Conclusions

### 🎯 Key Results

| Metric | Value |
|--------|-------|
| **ROC-AUC** | 0.9829 |
| **PR-AUC** | 0.8490 |
| **Fraud Recall** | 85.7% |
| **Precision** | 82.4% |
| **Cost Savings** | 85.1% |

### 🔑 Key Findings

1. **V14** is the strongest fraud predictor (0.159 importance)
2. **Amount_Zscore** (engineered feature) ranks #3 in importance
3. **Stacking Ensemble** achieves best overall performance
4. Model catches **84/98 fraud cases** with only **18 false alarms**

### 📚 Skills Demonstrated

- Imbalanced classification (SMOTE, class weights)
- Ensemble methods (Stacking)
- Feature engineering
- Fraud-specific evaluation metrics
- Cost-sensitive optimization

---

**Author**: Jeevan Arlagadda  
**Education**: MS Computer Science, University of Florida  
**Certification**: AWS Machine Learning Associate